# Crypto Currency Data From CoinGecko API

## Imported Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## Log Progress Fuction

In [ ]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./log_progress","a") as f:
        f.write(timestamp + ":" + message + '\n')

## Extract Fuction

In [ ]:
def extract(url):
    ''' This function aims to extract the required
    information from the CoinGecko and save it to a data frame. The
    function returns the data frame for further processing. '''
    # url = " https://api.coingecko.com/api/v3//coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st"
    params = {  
               'vs_currency': 'USD'
    }
    response = requests.get(url, params=params)
    if(response.status_code == 200):
        data = response.json()
        # print(data)
        df = pd.DataFrame(data)
        print(df)
    else:
        print("Failed to fetch data from API")

extract("https://api.coingecko.com/api/v3/coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st")